# Labo 2 PCD : Outliers detection

## Description des datasets

- Glass : Dataset contenant des descriptions de morceaux de verre recensés par la police scientifique
- Ionosphere : Données de radar répertoriées à Goose Bay au Canada.
- ALOI : Dataset contenant 110 images de 1000 objets de petite taille. Contient une variété d'images prises sous des angles et des conditions lumineuses différentes.

## Recherche des meilleurs paramètres

Pour les 3 datasets, nous avons recherché les meilleurs paramètres `n_neigbors` et `contamination` du modèle LocalOutlierFactor. Plus précisément, nous avons testé toutes les combinaisons des paramètres suivants :
- `n_neigbors` : Toutes les valeurs discrètes de 1 à 10
- `contamination` : Toutes les valeurs de 0.01 à 0.5 avec un pas de 0.4.

On aurait pu choisir des plus grandes intervalles et un pas plus petit pour le paramètre `contamination` mais ça prenait beaucoup de temps.

### Meilleurs paramètres, par dataset
Le code python ci-dessous permet de trouver ces paramètres en les testant de manière systématique pour chaque dataset. La sortie de notre programme présente les 5 meilleurs choix de paramètres pour chaque dataset avec le f1score, le recall ainsi que la précision du modèle obtenu avec ceux-ci.

![](results.png)



In [2]:
import numpy as np
import pandas as pnd
from scipy.io.arff import loadarff
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

###
### Déclarations de fonctions
###

def load_arff_df(path):
    raw_data = loadarff(path)
    df_data = pnd.DataFrame(raw_data[0])
    return df_data

def split_outlier(df):
    df_data_no_outlier = df.drop('outlier', axis=1)
    return df, df_data_no_outlier

def measure_lof_performance(df, lof_n_neighbors, lof_contamination):
    df_test, df_train = split_outlier(df)

    lof_model = LocalOutlierFactor(n_neighbors=lof_n_neighbors, contamination=lof_contamination)
    prediction = lof_model.fit_predict(df_train)

    outliers = df['outlier'].apply(lambda x: -1 if x == 'yes' else 1)

    return f1_score(outliers, prediction), recall_score(outliers, prediction), precision_score(outliers, prediction)

###
### Entrée du programme
###

dataframes = [
    ["Glass", load_arff_df("./data/Glass/Glass_withoutdupl_norm.arff")],
    ["Ionosphere", load_arff_df("./data/Ionosphere/Ionosphere_withoutdupl_norm.arff")],
    ["ALOI", load_arff_df("./data/ALOI/ALOI_withoutdupl_norm.arff")]
]

# pour chaque dataframe...
for name, df in dataframes :
    # on calcule le score du modèle, pour un grand nombre de paramètres différents :
    # - nombre de voisins : de 1 à 30
    # - "contamination" : de 0.1 à 0.5, avec un step de 0.05
    results = pnd.DataFrame(columns=['f1-score', 'recall', 'precision', 'neighbors','contamination'])
    index = 0
    for neighbors in range(1, 10):
        for i,contamination in enumerate(np.arange(0.01, 0.5, 0.4)):
            # mesure de la performance pour les paramètres actuels
            f1score, recall, precision = measure_lof_performance(df, neighbors, contamination)

            rslt = pnd.Series({
                'f1-score': f1score, 
                'recall': recall,
                'precision': precision,
                'neighbors' : neighbors, 
                'contamination' : contamination
                })
            
            index = index + 1
            results.loc[index] = [f1score, recall, precision, neighbors, contamination]
            #results.loc[len(results.index)] = [f1score, recall, precision, neighbors, contamination]
    sorted_results = results.sort_values(by=['f1-score'], ascending=False)
    print("Top 5 meilleurs résultats pour le dataframe ", name, " :\n", sorted_results.head())

Top 5 meilleurs résultats pour le dataframe  Glass  :
     f1-score    recall  precision  neighbors  contamination
9   0.995305  0.990654        1.0        5.0           0.01
1   0.992941  0.985981        1.0        1.0           0.01
13  0.992941  0.985981        1.0        7.0           0.01
3   0.992941  0.985981        1.0        2.0           0.01
17  0.992941  0.985981        1.0        9.0           0.01
Top 5 meilleurs résultats pour le dataframe  Ionosphere  :
     f1-score    recall  precision  neighbors  contamination
1   0.994269  0.988604        1.0        1.0           0.01
13  0.994269  0.988604        1.0        7.0           0.01
3   0.994269  0.988604        1.0        2.0           0.01
17  0.994269  0.988604        1.0        9.0           0.01
5   0.994269  0.988604        1.0        3.0           0.01
Top 5 meilleurs résultats pour le dataframe  ALOI  :
     f1-score    recall  precision  neighbors  contamination
1   0.994968  0.989987        1.0        1.0       